In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from statistics import mean

In [11]:

data = pd.read_csv("clean_weather.csv", index_col=0)
data

,tmax,tmin,rain,tmax_tomorrow
1970-01-01,60.0,35.0,0.0,52.0
1970-01-02,52.0,39.0,0.0,52.0
1970-01-03,52.0,35.0,0.0,53.0
1970-01-04,53.0,36.0,0.0,52.0
1970-01-05,52.0,35.0,0.0,50.0
...,...,...,...,...
2022-11-22,62.0,35.0,0.0,67.0
2022-11-23,67.0,38.0,0.0,66.0
2022-11-24,66.0,41.0,0.0,70.0
2022-11-25,70.0,39.0,0.0,62.0


In [15]:
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

In [17]:
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

C:\Users\ASIM\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [22]:
train_x.shape
#train_y.shape
#valid_x.shape
#valid_y.shape
#test_x.shape
#test_y.shape

(9456, 3)

In [6]:
def mse(actual, predicted):
    return np.mean((actual-predicted)**2)

def mse_grad(actual, predicted):
    return (predicted - actual)

In [7]:
def init_layers(inputs):
    layers = []
    for i in range(1, len(inputs)):
        layers.append([
            np.random.rand(inputs[i-1], inputs[i]) / 5 - .1,
            np.ones((1,inputs[i]))
        ])
    return layers

In [8]:
def forward(batch, layers):
    hidden = [batch.copy()]
    for i in range(len(layers)):
        batch = np.matmul(batch, layers[i][0]) + layers[i][1]
        if i < len(layers) - 1:
            batch = np.maximum(batch, 0)
        hidden.append(batch.copy())

    return batch, hidden

## Hidden

1,3
1,10
1,10

## Output
1,1

## Weights

3,10
10,10
10,1

## Backwards

### Iter 1 - layer 3

* grad 1,1
* w_grad 1,1 * 1,10 = 1,10 .T = 10,1
* grad 10,1 * 1,1 = 10,1 .T = 1,10

### Iter 2 - layer 2

* grad 10,1
* w_grad 10,1 * 1,10 = 10,10
* grad 10,10 10,1 = 10,1 . T = 1,10

## Iter 3 - layer 1

* grad 10,1
* w_grad 10,1 1,3 = 10,3 .T = 3,10
* grad 3,10 10,1 = 3,1 .T = 1,3

In [9]:
def backward(layers, hidden, grad, lr):
    for i in range(len(layers)-1, -1, -1):
        if i != len(layers) - 1:
            grad = np.multiply(grad, np.heaviside(hidden[i+1], 0))

        grad = grad.T
        w_grad = np.matmul(grad, hidden[i]).T
        b_grad = np.mean(grad.T, axis=0)

        layers[i][0] -= (w_grad + layers[i][0] * .01) * lr
        layers[i][1] -= b_grad * lr
        grad = np.matmul(layers[i][0], grad).T
    return layers

In [10]:
layer_conf = [3,10,10,1]
lr = 1e-6
epochs=10
batch_size = 8

layers = init_layers(layer_conf)

for epoch in range(epochs):
    epoch_loss = []

    for i in range(0, train_x.shape[0], batch_size):
        x_batch = train_x[i:(i+batch_size)]
        y_batch = train_y[i:(i+batch_size)]
        pred, hidden = forward(x_batch, layers)

        loss = mse_grad(y_batch, pred)
        epoch_loss.append(np.mean(loss ** 2))

        layers = backward(layers, hidden, loss, lr)
    

    valid_preds, _ = forward(valid_x, layers)

    print(f"Epoch: {epoch} Train MSE: {mean(epoch_loss)} Valid MSE: {mse(valid_preds,valid_y)}")

NameError: name 'train_x' is not defined